In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
#matplotlib figure size
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
from custom_imports import *
from mri_project.utility import dfe
__spec__ = 'Mardas'

In [2]:
files = glob.glob("../data/images_with_predictions_v04/*data")
file = np.random.choice(files)

In [3]:
from uuid import uuid4
predicted = []
traced = []
def abbas(x):
    return np.uint8((x-x.min())/(x.max()-x.min())*255)
def mardas(file, index=-1):
    print(index, file)
    data = joblib.load(file)
    data.get_traced_contours(89.9)
    data.get_predicted_contours(89.9)
    dic = {
        'raw': data.raw_image,
        'traced': data.traced_image,
        'predicted': data.predicted,
        'traced_lever_arms_90': data.traced_lever_arm_images[89.9]+150,
        'predicted_lever_arms_90': data.predicted_lever_arm_images[89.9]+150,
    }
    d, f, e = dfe(data.id)
    dr = d.replace('mri_analysis', 'mri_analysis_behnam')\
        .replace(" Set#", '/Set').replace(' ', '__')\
        .replace('..', 'blahblah').replace('.', '_').replace('blahblah', '..')
    if not os.path.exists(dr):
        os.makedirs(dr)
    for k, v in dic.items():
        # plt.imshow(v)
        # plt.axis('off')
        # plt.savefig(f"{dr}/{k}.png", dpi=150, bbox_inches=0)
        f_name = f"{dr}/{k}_{f}.png"
        cv2.imwrite(f_name, abbas(v))
    data.get_contour_areas()
    data.get_contour_centers() 
    return data.id, data.traced_features, data.predicted_features

In [7]:
for index, file in enumerate(files):
    if index % 40 == 0:
        print(index, file)
    print(mardas(file))
    break

0 ../data/images_with_predictions_v04/1-CO5-3:16RepPreRaw.data
-1 ../data/images_with_predictions_v04/1-CO5-3:16RepPreRaw.data
({'lever_arm_89.9': [0.025155503829758986, 5.372914585877734, 6.188284005693532, 2.7807533955590995, 5.450528064010565, 3.728793271536483, 4.525621173609459, 3.423485122181562, 3.728366759992871, 10.911840187792372, 11.964729264187469], 'area': [12.057417322140703, 4.00848994939814, 3.1356262072105823, 5.412709953657803, 5.615411167661813, 22.18366492608016, 23.462298308619946, 8.787024184604991, 9.492439097832715, 30.45145085597198, 28.679284082813734], 'center': [array([707.92922374, 707.56392694]), array([905.46116505, 821.77184466]), array([478.8852459, 840.6557377]), array([809.12886598, 431.3556701 ]), array([505.4       , 434.72608696]), array([844.9254386 , 897.65570175]), array([540.352657  , 906.57971014]), array([833.35474006, 723.01223242]), array([569.46368715, 730.27932961]), array([1109.59655597,  677.80196802]), array([265.52941176, 723.38316993

In [4]:
import dask.bag as db

In [6]:
res = db.from_sequence(files).map(mardas).compute()

In [39]:
from mri_project.muscle_detector import nine_to_11_dict
def clean_features(feat):
    out = dict(enumerate(feat))
    if len(feat) == 9:
        out = {nine_to_11_dict[k]: v for k, v in out.items()}
    return list(out.items())

In [84]:
traced_features = pd.DataFrame([x[1] for x in res]).assign(**{'id': [x[0] for x in res]})
tmp_ = traced_features.set_index('id').unstack().to_frame()
tmp_[0] = tmp_[0].map(clean_features)
tmp_ = tmp_.explode(0)
tmp_[['muscle_number', 'value']] =  pd.DataFrame(tmp_[0].tolist(), index=tmp_.index)
traced_df = tmp_.drop(0, axis=1)
traced_df = traced_df.reset_index().set_index(['level_0', 'id', 'muscle_number']).unstack(0)['value'].reset_index()

In [85]:
predicted_features = pd.DataFrame([x[2] for x in res]).assign(**{'id': [x[0] for x in res]})
tmp_ = predicted_features.set_index('id').unstack().to_frame()
tmp_[0] = tmp_[0].map(clean_features)
tmp_ = tmp_.explode(0)
tmp_[['muscle_number', 'value']] =  pd.DataFrame(tmp_[0].tolist(), index=tmp_.index)
predicted_df = tmp_.drop(0, axis=1)
predicted_df = predicted_df.reset_index().set_index(['level_0', 'id', 'muscle_number']).unstack(0)['value'].reset_index()

In [87]:
predicted_df.shape, traced_df.shape

((8400, 5), (8432, 5))

In [91]:
df = predicted_df.merge(traced_df, on=['id', 'muscle_number'], suffixes=["_predicted", '_traced'])

In [92]:
df.to_csv("../data/Fehmi/all_features.csv", index=False)

In [93]:
df

level_0,id,muscle_number,area_predicted,center_predicted,lever_arm_89.9_predicted,area_traced,center_traced,lever_arm_89.9_traced
0,../data/mri_analysis/1.Multigroup Set#1_DONE/C...,0,16.1074,"[716.3617021276596, 710.1306990881459]",0.00979603,12.317,"[704.6863753213368, 701.8071979434447]",0.0185627
1,../data/mri_analysis/1.Multigroup Set#1_DONE/C...,1,7.20287,"[947.2331606217616, 827.1813471502591]",6.26093,5.47881,"[587.8949579831933, 737.7142857142857]",3.14821
2,../data/mri_analysis/1.Multigroup Set#1_DONE/C...,2,8.64308,"[470.5659340659341, 819.3516483516484]",6.65649,5.70134,"[827.2916666666666, 739.4541666666667]",3.33941
3,../data/mri_analysis/1.Multigroup Set#1_DONE/C...,3,14.2926,"[876.2478386167147, 448.64265129682997]",4.35512,2.12653,"[502.75132275132273, 812.2169312169312]",5.45915
4,../data/mri_analysis/1.Multigroup Set#1_DONE/C...,4,12.1503,"[540.6023622047244, 446.9488188976378]",4.74087,26.8763,"[554.2135416666666, 890.984375]",4.06823
...,...,...,...,...,...,...,...,...
8363,../data/mri_analysis/4.Contiguous Set#2/Weight...,6,23.2587,"[813.6892655367232, 1019.2730696798493]",3.36616,18.3015,"[769.8281505728314, 1025.4926350245498]",4.40486
8364,../data/mri_analysis/4.Contiguous Set#2/Weight...,7,9.0568,"[1159.73440285205, 803.3351158645277]",3.30863,5.50711,"[1162.9712460063897, 817.3450479233227]",3.17714
8365,../data/mri_analysis/4.Contiguous Set#2/Weight...,8,8.77558,"[840.8084714548803, 802.4475138121547]",2.83634,4.50121,"[834.8545034642033, 825.635103926097]",3.14522
8366,../data/mri_analysis/4.Contiguous Set#2/Weight...,9,31.1198,"[1515.9841849148418, 682.7232360097323]",10.1768,17.4392,"[1531.6973451327433, 675.7619469026548]",10.2864


In [10]:
df = pd.read_csv("../data/prediction_stats_v01.csv")

In [13]:
df.metric.unique()

array(['area_ratio', 'center_distance', 'lever_arm_89.8_distance',
       'total_accuracy', 'accuracy', 'precision', 'recall'], dtype=object)

In [17]:
df

,Unnamed: 0,metric,id,muscle,value,n_muscles
0,0,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,11,1.575099,9
1,1,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,11,3.791567,9
2,2,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,11,2.927293,9
3,3,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,11,1.532780,9
4,4,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,11,1.511721,9
...,...,...,...,...,...,...
46871,30515,recall,../data/mri_analysis/4.Contiguous Set#2/Weight...,7,0.975620,11
46872,30516,recall,../data/mri_analysis/4.Contiguous Set#2/Weight...,8,0.954185,11
46873,30517,recall,../data/mri_analysis/4.Contiguous Set#2/Weight...,9,0.913792,11
46874,30518,recall,../data/mri_analysis/4.Contiguous Set#2/Weight...,10,0.998315,11
